In [38]:
import sys
from pathlib import Path
from finbert.finbert import predict
from transformers import AutoModelForSequenceClassification
import nltk
import pandas as pd
from bs4 import BeautifulSoup
import json 
import requests

In [39]:
pd.options.display.max_colwidth = 500

In [40]:
#f = pd.read_csv("2.csv")
#f.drop(['logit'],axis=1,inplace=True)
#f

In [41]:
#야후 파이낸스를 통해 특정 종목에 대한 최신 뉴스 기사들의 id 수집
def getStockNews(code):
    url = "https://apidojo-yahoo-finance-v1.p.rapidapi.com/news/v2/list"
    querystring = {"region":"US","snippetCount":"30","s":code}
    payload = ""
    headers = {
        'content-type': "text/plain",
        'x-rapidapi-host': "apidojo-yahoo-finance-v1.p.rapidapi.com",
        'x-rapidapi-key': "f977847401mshd7a2fe4cba6191fp1ae565jsnf4676589249d"
        }

    response = requests.request("POST", url, data=payload, headers=headers, params=querystring)
    json_data = json.loads(response.text)
    result = []
    for ele in json_data["data"]["main"]["stream"]:
        if len(ele["content"]["finance"]["stockTickers"]) == 1 :#해당 종목에 대한 기사인지 확인
            result.append(ele["content"]["id"])
    return result

In [42]:
#수집한 뉴스 기사 내용 반환
def getNewsContent(ID):
    url = "https://apidojo-yahoo-finance-v1.p.rapidapi.com/news/v2/get-details"

    querystring = {"uuid":ID,"region":"US"}
    headers = {
        'x-rapidapi-host': "apidojo-yahoo-finance-v1.p.rapidapi.com",
        'x-rapidapi-key': "f977847401mshd7a2fe4cba6191fp1ae565jsnf4676589249d"
        }

    response = requests.request("GET", url, headers=headers, params=querystring)
    json_data1 = json.loads(response.text)
    markup= json_data1["data"]['contents'][0]['content']['body']['markup']
    bs = BeautifulSoup(markup,'html.parser')
    result = bs.find('div', class_ ="caas-body")
    return result.get_text()

In [43]:
#심볼에 해당하는 주식 기사들의 감성비율(긍정/부정)의 판다스 프레임 반환
def sentimentAnalysis(code):
    news_senti_list = []
    dic = {0:"positive",1:"negative"}
    
    #특정 심볼에 해당하는 종목 기사들 수집
    news = getStockNews(code)
    #기사들의 주요 감성 분석 결과 배열에 저장
    for i in range(len(news)):
        one_news_text = getNewsContent(news[i])
        project_dir = Path.cwd()
        cl_path = project_dir/'models'/'sentiment'/'finbert'
        model = AutoModelForSequenceClassification.from_pretrained(cl_path, cache_dir=None, num_labels=3)
        new_sentences_senti = predict(one_news_text,model)
        #1개의 기사에 대한 주요 감성 확인
        senti = [0,0]
        for j in range(len(new_sentences_senti)):
            if new_sentences_senti["prediction"][j] == "positive":
                senti[0] += 1
            elif new_sentences_senti["prediction"][j] == "negative":
                senti[1] += 1
        news_senti_list.append(dic[senti.index(max(senti))])
    #[심볼,긍정기사비율,부정기사비율] 반환    
    ans = [code,news_senti_list.count("positive")/len(news_senti_list)*100,news_senti_list.count("negative")/len(news_senti_list)*100]    
    return ans

In [44]:
lists = ["MSFT", "ORCL", "AAPL", "IBM"]
code_list = ["MSFT", "ORCL", "AAPL", "IBM", "GOOGL", "FB", "NFLX", "DIS", "AMZN", "TSLA", "SBUX", "NKE", "WMT", "COST", "KO", "PEP", "V", "PYPL", "BAC", "C", "WFC","JNJ","PFE", "UNH", "AMGN", "LLY","HON", "UNP", "MMM", "TT", "LMT","AMT", "EQIX", "PLD", "O"]
code_list = ["AMGN","LLY","HON", "UNP", "MMM", "TT", "LMT","AMT", "EQIX", "PLD", "O"]

In [45]:
template = {'Stock' : [], 'positive' : [], 'negative': [] }
result = pd.DataFrame(template)

for code in code_list:
    senti_analysis = sentimentAnalysis(code)
    update = pd.DataFrame({'Stock' : [senti_analysis[0]], 'positive' : [senti_analysis[1]], 'negative': [senti_analysis[2]]})
    result = pd.concat([result,update])
result

09/19/2021 10:40:58 - INFO - finbert.utils -   *** Example ***
09/19/2021 10:40:58 - INFO - finbert.utils -   guid: 0
09/19/2021 10:40:58 - INFO - finbert.utils -   tokens: [CLS] while some investors are already well verse ##d in financial metric ##s ( hat tip ) , this article is for those who would like to learn about return on equity ( roe ) and why it is important . [SEP]
09/19/2021 10:40:58 - INFO - finbert.utils -   input_ids: 101 2096 2070 9387 2024 2525 2092 7893 2094 1999 3361 12046 2015 1006 6045 5955 1007 1010 2023 3720 2003 2005 2216 2040 2052 2066 2000 4553 2055 2709 2006 10067 1006 20944 1007 1998 2339 2009 2003 2590 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
09/19/2021 10:40:58 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
09/19/2021 10:40:58 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

09/19/2021 10:41:16 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
09/19/2021 10:41:16 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
09/19/2021 10:41:16 - INFO - finbert.utils -   label: None (id = 9090)
09/19/2021 10:41:17 - INFO - root -   tensor([[ 0.4619, -1.3658,  0.4990],
        [ 1.8686, -2.1589,  0.3407],
        [-1.2648, -1.1262,  2.7124],
        [-0.8905, -1.4056,  2.8952]])
09/19/2021 10:41:28 - INFO - finbert.utils -   *** Example ***
09/19/2021 10:41:28 - INFO - finbert.utils -   guid: 0
09/19/2021 10:41:28 - INFO - finbert.utils -   tokens: [CLS] new data at es ##mo 2021 support initiation of phase 3 trial of lu ##ma ) today announced the first combination study results from the phase 1b / 2 code ##break 101 study , the most

09/19/2021 10:41:32 - INFO - finbert.utils -   input_ids: 101 5678 1010 2572 6914 2038 7864 2019 5003 2050 1999 1996 2647 2586 1010 2900 23890 10760 3642 23890 6612 2458 2565 2005 2572 6914 1005 1055 4319 22768 8180 12322 2003 2881 2000 7438 5022 2007 2019 3935 5024 13656 2007 1996 1047 8180 1043 12521 2278 16221 1998 4769 1996 2146 24911 4895 11368 2966 2342 2005 2122 25409 1012 102
09/19/2021 10:41:32 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
09/19/2021 10:41:32 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
09/19/2021 10:41:32 - INFO - finbert.utils -   label: None (id = 9090)
09/19/2021 10:41:33 - INFO - root -   tensor([[ 1.2339, -2.7726,  1.3320],
        [ 0.4137, -2.5731,  2.1139],
        [-0.4153, -2.0991,  2.6273],
        [-0

09/19/2021 10:41:36 - INFO - finbert.utils -   label: None (id = 9090)
09/19/2021 10:41:37 - INFO - root -   tensor([[-0.8398, -1.4006,  2.7819],
        [-0.5556, -1.5776,  2.5557],
        [-1.1490, -0.2198,  1.0019],
        [ 0.2400, -2.2731,  1.9504],
        [-0.7939,  1.4635, -1.3978]])
09/19/2021 10:41:37 - INFO - finbert.utils -   *** Example ***
09/19/2021 10:41:37 - INFO - finbert.utils -   guid: 0
09/19/2021 10:41:37 - INFO - finbert.utils -   tokens: [CLS] monitor patients for h ##yp ##oma ##gne ##se ##mia and h ##yp ##oca ##lce ##mia prior to initiating ve ##ct ##ib ##ix ##® treatment , periodically during ve ##ct ##ib ##ix ##® treatment , and for up to 8 weeks after the completion of treatment . [SEP]
09/19/2021 10:41:37 - INFO - finbert.utils -   input_ids: 101 8080 5022 2005 1044 22571 9626 10177 3366 10092 1998 1044 22571 24755 23314 10092 3188 2000 26616 2310 6593 12322 7646 29656 3949 1010 18043 2076 2310 6593 12322 7646 29656 3949 1010 1998 2005 2039 2000 1022 3134

09/19/2021 10:41:40 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
09/19/2021 10:41:40 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
09/19/2021 10:41:40 - INFO - finbert.utils -   label: None (id = 9090)
09/19/2021 10:41:41 - INFO - root -   tensor([[-1.9509,  1.5405, -0.2572],
        [-1.7197, -0.0175,  1.4663],
        [-0.1031, -2.3458,  2.1061],
        [-1.1166, -0.4236,  0.8573],
        [-0.0114, -2.7111,  2.8543]])
09/19/2021 10:41:41 - INFO - finbert.utils -   *** Example ***
09/19/2021 10:41:41 - INFO - finbert.utils -   guid: 0
09/19/2021 10:41:41 - INFO - finbert.utils -   tokens: [CLS] product candidates that are derived from relationships may be subject to disputes between the parties or may prove to be not as effective or as 

...

09/19/2021 11:19:02 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
09/19/2021 11:19:02 - INFO - finbert.utils -   label: None (id = 9090)
09/19/2021 11:19:03 - INFO - root -   tensor([[ 1.7754, -0.1367, -1.8180],
        [ 2.3167, -1.2250, -0.9202],
        [ 2.6241, -1.0993, -1.1139],
        [ 2.5986, -1.3464, -0.8115],
        [ 0.7852, -1.5911,  0.4187]])
09/19/2021 11:19:03 - INFO - finbert.utils -   *** Example ***
09/19/2021 11:19:03 - INFO - finbert.utils -   guid: 0
09/19/2021 11:19:03 - INFO - finbert.utils -   tokens: [CLS] these recent revisions tend to reflect the evolving nature of short - term business trends . [SEP]
09/19/2021 11:19:03 - INFO - finbert.utils -   input_ids: 101 2122 3522 24699 7166 2000 8339 1996 20607 3267 1997 2460 1011 2744 2449 12878 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

09/19/2021 11:19:49 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
09/19/2021 11:19:49 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
09/19/2021 11:19:49 - INFO - finbert.utils -   label: None (id = 9090)
09/19/2021 11:19:50 - INFO - root -   tensor([[-1.8378,  1.4629, -0.1214],
        [ 1.1724, -0.0447, -1.6558],
        [ 1.9256, -1.4064, -0.4590],
        [ 2.4558, -0.9607, -1.2250],
        [ 2.1612, -1.3306, -0.5900]])
09/19/2021 11:19:50 - INFO - finbert.utils -   *** Example ***
09/19/2021 11:19:50 - INFO - finbert.utils -   guid: 0
09/19/2021 11:19:50 - INFO - finbert.utils -   tokens: [CLS] the top - line figure also climbed 12 % year on year . real ##ty income also app ##rise ##d of its rental receipts through jun 30 , 2021 , and n

09/19/2021 11:20:15 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
09/19/2021 11:20:15 - INFO - finbert.utils -   label: None (id = 9090)
09/19/2021 11:20:16 - INFO - root -   tensor([[ 0.0591, -2.7969,  2.6896],
        [-0.5988, -1.7563,  2.2563],
        [-0.8170, -0.1153,  0.6580],
        [-1.0559, -0.6777,  1.0201],
        [-0.1814, -2.3675,  2.5715]])
09/19/2021 11:20:16 - INFO - finbert.utils -   *** Example ***
09/19/2021 11:20:16 - INFO - finbert.utils -   guid: 0
09/19/2021 11:20:16 - INFO - finbert.utils -   tokens: [CLS] we discount the terminal cash flows to today ' s value at the cost of next ten years plus the discount ##ed terminal value , which results in the total equity value , which in this case is us $ 48 ##b . to get the intrinsic value per share , we divide this by the total number of shares outstanding . [SEP]
09/19/2021 11:20:16 - INFO -

09/19/2021 11:20:44 - INFO - finbert.utils -   *** Example ***
09/19/2021 11:20:44 - INFO - finbert.utils -   guid: 0
09/19/2021 11:20:44 - INFO - finbert.utils -   tokens: [CLS] san diego , aug . 17 , 2021 / pr ##ne ##ws ##wire / - - real ##ty income corporation ( real ##ty income , ny ##se : o ) , the monthly divide ##nd company ##® , today announced that its board of directors has declared the 61 ##4th consecutive common stock monthly divide ##nd . [SEP]
09/19/2021 11:20:44 - INFO - finbert.utils -   input_ids: 101 2624 5277 1010 15476 1012 2459 1010 25682 1013 10975 2638 9333 20357 1013 1011 1011 2613 3723 3318 3840 1006 2613 3723 3318 1010 6396 3366 1024 1051 1007 1010 1996 7058 11443 4859 2194 29656 1010 2651 2623 2008 2049 2604 1997 5501 2038 4161 1996 6079 26724 5486 2691 4518 7058 11443 4859 1012 102 0 0 0 0 0
09/19/2021 11:20:44 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 

,Stock,positive,negative
0,AMGN,84.210526,15.789474
0,LLY,86.666667,13.333333
0,HON,100.000000,0.000000
0,UNP,87.500000,12.500000
0,MMM,84.210526,15.789474
0,TT,100.000000,0.000000
0,LMT,100.000000,0.000000
0,AMT,100.000000,0.000000
0,EQIX,100.000000,0.000000
0,PLD,100.000000,0.000000


In [83]:
result[result["Stock"]=="MSFT"]["positive"]

0    100.0
Name: positive, dtype: float64

In [46]:
result

,Stock,positive,negative
0,AMGN,84.210526,15.789474
0,LLY,86.666667,13.333333
0,HON,100.000000,0.000000
0,UNP,87.500000,12.500000
0,MMM,84.210526,15.789474
0,TT,100.000000,0.000000
0,LMT,100.000000,0.000000
0,AMT,100.000000,0.000000
0,EQIX,100.000000,0.000000
0,PLD,100.000000,0.000000


In [47]:
result.to_csv("news_senti_ratio.csv",index=False)